In [3]:
%pip install assemblyai

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



  Obtaining dependency information for assemblyai from https://files.pythonhosted.org/packages/8d/17/a8880e6032f40c1431549f0a3bd058f6e5e19d7938419955cd7bd3feac9d/assemblyai-0.24.0-py3-none-any.whl.metadata
  Obtaining dependency information for httpx>=0.19.0 from https://files.pythonhosted.org/packages/41/7b/ddacf6dcebb42466abd03f368782142baa82e08fc0c1f8eaa05b4bae87d5/httpx-0.27.0-py3-none-any.whl.metadata
  Obtaining dependency information for pydantic!=1.10.7,>=1.7.0 from https://files.pythonhosted.org/packages/e5/f3/8296f550276194a58c5500d55b19a27ae0a5a3a51ffef66710c58544b32d/pydantic-2.6.4-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/85.1 kB ? eta -:--:--
     ---------------------------------------- 0.0/85.1 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/85.1 kB ? eta -:--:--
     ------------------- -------------------- 41.0/85.1 kB ? eta -:--:--
     ---------------------- --------------- 51.2/85.1 kB 440.4 kB/s eta 0:00:01
  

In [6]:
%pip install pyaudio

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



  Obtaining dependency information for pyaudio from https://files.pythonhosted.org/packages/27/bc/719d140ee63cf4b0725016531d36743a797ffdbab85e8536922902c9349a/PyAudio-0.2.14-cp310-cp310-win_amd64.whl.metadata
   ---------------------------------------- 0.0/164.1 kB ? eta -:--:--
   --------- ----------------------------- 41.0/164.1 kB 991.0 kB/s eta 0:00:01
   --------- ----------------------------- 41.0/164.1 kB 991.0 kB/s eta 0:00:01
   ----------------- --------------------- 71.7/164.1 kB 653.6 kB/s eta 0:00:01
   --------------------- ----------------- 92.2/164.1 kB 581.0 kB/s eta 0:00:01
   -------------------------- ----------- 112.6/164.1 kB 504.4 kB/s eta 0:00:01
   ---------------------------- --------- 122.9/164.1 kB 479.3 kB/s eta 0:00:01
   -------------------------------------- 164.1/164.1 kB 517.9 kB/s eta 0:00:00


In [7]:
%pip install websockets

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import websockets
import asyncio
import base64
import json
# from configure import auth_key

import pyaudio
 
FRAMES_PER_BUFFER = 3200
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
p = pyaudio.PyAudio()
 
# starts recording
stream = p.open(
	format=FORMAT,
	channels=CHANNELS,
	rate=RATE,
	input=True,
	frames_per_buffer=FRAMES_PER_BUFFER
)
 
# the AssemblyAI endpoint we're going to hit
URL = "wss://api.assemblyai.com/v2/realtime/ws?sample_rate=16000"
 
async def send_receive():

	print(f'Connecting websocket to url ${URL}')

	async with websockets.connect(
		URL,
		extra_headers=(("Authorization", auth_key),),
		ping_interval=5,
		ping_timeout=20
	) as _ws:

		await asyncio.sleep(0.1)
		print("Receiving SessionBegins ...")

		session_begins = await _ws.recv()
		print(session_begins)
		print("Sending messages ...")


		async def send():
			while True:
				try:
					data = stream.read(FRAMES_PER_BUFFER)
					data = base64.b64encode(data).decode("utf-8")
					json_data = json.dumps({"audio_data":str(data)})
					await _ws.send(json_data)

				except websockets.exceptions.ConnectionClosedError as e:
					print(e)
					assert e.code == 4008
					break

				except Exception as e:
					assert False, "Not a websocket 4008 error"

				await asyncio.sleep(0.01)
		  
			return True
	  

		async def receive():
			while True:
				try:
					result_str = await _ws.recv()
					print(json.loads(result_str)['text'])

				except websockets.exceptions.ConnectionClosedError as e:
					print(e)
					assert e.code == 4008
					break

				except Exception as e:
					assert False, "Not a websocket 4008 error"
	  
		send_result, receive_result = await asyncio.gather(send(), receive())

while True:
	asyncio.run(send_receive())

In [4]:
import assemblyai as aai

aai.settings.api_key = "a220a8023ff44b8b9ee8d7498ac1ba64"
transcriber = aai.Transcriber()

transcript = transcriber.transcribe("https://storage.googleapis.com/aai-web-samples/news.mp4")
# transcript = transcriber.transcribe("./my-local-audio-file.wav")

print(transcript.text)

I'm David Curley at the Smithsonian Air and Space Museum, where we are marking 50 years since man landed and walked on the moon in a lander just like this one. We are going to show you some of the actual ABC News coverage from 50 years ago. During that eight day mission of this remarkable achievement, Apollo Eleven's lander, the Eagle, would be the first man craft to land on the moon. For training, NASA came up with an unusual contraption. Neil Armstrong actually had to eject from it once, and then he had a couple of successful flights. ABC News anchor at the time, 50 years ago, Frank Reynolds with a look at that unusual trainer. Apollo Eleven Commander Neil Armstrong is at the controls of a lunar landing training vehicle, testing the reaction control jets. These thrusters stabilize the LEM during landing and takeoff. The LLTV is designed to simulate the behavior of the LEM as it lands in the moon's gravity. Lunar gravity is one 6th that of the Earth's. Neil ArmstroNG flew one of these